# TiDB Vector

> [TiDB](https://github.com/pingcap/tidb) is an open-source, cloud-native, distributed, MySQL-Compatible database for elastic scale and real-time analytics.

In its latest version (insert version number here), TiDB introduces support for vector search. This notebook provides a detailed guide on utilizing the TiDB Vector functionality, showcasing its features and practical applications.

## Setting up environments

Begin by installing the necessary packages.

In [ ]:
%pip install langchain
%pip install langchain-openai

Configure both the OpenAI and TiDB host settings that you will need

In [2]:
# Here we useimport getpass
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")
tidb_connection_url = getpass.getpass(
    "TiDB connection URL (format - mysql+pymysql://root@127.0.0.1:4000/test): "
)

Prepare the following data

In [3]:
from langchain.docstore.document import Document
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores.tidb_vector import TiDBVector
from langchain_openai import OpenAIEmbeddings

In [4]:
loader = TextLoader("../../modules/state_of_the_union.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()

## Semantic similarity search

TiDB supports both cosine and Euclidean distances ('cosine', 'l2'), with 'cosine' being the default choice.

The code snippet below creates a table named 'COLLECTION_NAME' in TiDB, optimized for vector searching. Upon successful execution of this code, you will be able to view and access the 'collection name' table directly within your TiDB database environment

In [5]:
COLLECTION_NAME = "state_of_the_union_test"
db = TiDBVector.from_documents(
    documents=docs,
    embedding=embeddings,
    collection_name=COLLECTION_NAME,
    connection_string=tidb_connection_url,
    distance_strategy="cosine",  # default, another option is "l2"
)

In [6]:
query = "What did the president say about Ketanji Brown Jackson"
docs_with_score = db.similarity_search_with_score(query, k=3)

Please note that a lower cosine distance indicates higher similarity.

In [7]:
for doc, score in docs_with_score:
    print("-" * 80)
    print("Score: ", score)
    print(doc.page_content)
    print("-" * 80)

--------------------------------------------------------------------------------
Score:  0.18459301498220004
Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.
--------------------------------------------------------------------------------
----------------------

Additionally, the similarity_search_with_relevance_scores method can be used to obtain relevance scores, where a higher score indicates greater similarity.

In [8]:
docs_with_relevance_score = db.similarity_search_with_relevance_scores(query, k=2)
for doc, score in docs_with_relevance_score:
    print("-" * 80)
    print("Score: ", score)
    print(doc.page_content)
    print("-" * 80)

--------------------------------------------------------------------------------
Score:  0.8154069850178
Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.
--------------------------------------------------------------------------------
--------------------------

# Filter with metadata

perform searches using metadata filters to retrieve a specific number of nearest-neighbor results that align with the applied filters.

## Supported metadata types

Each vector in the collection can be paired with metadata, structured as key-value pairs within a JSON object. The keys are strings, and the values can be of the following types:

- String
- Number (integer or floating point)
- Booleans (true, false)

For instance, consider the following valid metadata payloads:

```json
{
    "page": 12,
    "book_tile": "Siddhartha"
}
```

## Metadata filter syntax

The available filters include:

- $or - Selects vectors that meet any one of the given conditions.
- $and - Selects vectors that meet all of the given conditions.
- $eq - Equal to
- $ne - Not equal to
- $gt - Greater than
- $gte - Greater than or equal to
- $lt - Less than
- $lte - Less than or equal to
- $in - In array
- $nin - Not in array

Assuming one vector with metada:
```json
{
    "page": 12,
    "book_tile": "Siddhartha"
}
```

The following metadata filters will match the vector

```json
{"page": 12}

{"page":{"$eq": 12}}

{"page":{"$in": [11, 12, 13]}}

{"page":{"$nin": [13]}}

{"page":{"$lt": 11}}

{
    "$or": [{"page": 11}, {"page": 12}],
    "$or": [{"page": 12}, {"page": 13}],
}
```

Please note that each key-value pair in the metadata filters is treated as a separate filter clause, and these clauses are combined using the AND logical operator.

In [9]:
db.add_texts(
    texts = [
        "TiDB Vector offers advanced, high-speed vector processing capabilities, enhancing AI workflows with efficient data handling and analytics support.",
        "TiDB Vector, starting as low as $10 per month for basic usage",
    ],
    metadatas = [
        {"title": "TiDB Vector functionality"},
        {"title": "TiDB Vector Pricing"},
    ]
)

[UUID('fd5ffb4f-9cc4-4402-85a6-c82fc2fa3b3e'),
 UUID('5aae9127-c154-48d0-9c3d-c2ccae816ab0')]

In [10]:
docs_with_relevance_score = db.similarity_search_with_score("Introduction to TiDB Vector", filter={"title": "TiDB Vector functionality"}, k=4)
for doc, score in docs_with_relevance_score:
    print("-" * 80)
    print("Score: ", score)
    print(doc.page_content)
    print("-" * 80)

--------------------------------------------------------------------------------
Score:  0.12761409169211535
TiDB Vector offers advanced, high-speed vector processing capabilities, enhancing AI workflows with efficient data handling and analytics support.
--------------------------------------------------------------------------------


### Using as a Retriever

In Langchain, a retriever is an interface that retrieves documents in response to an unstructured query, offering a broader functionality than a vector store. The code below demonstrates how to utilize TiDB Vector as a retriever.

In [11]:
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 3, "score_threshold": 0.8},
)
docs_retrieved = retriever.get_relevant_documents(query)
for doc in docs_retrieved:
    print("-" * 80)
    print(doc.page_content)
    print("-" * 80)

--------------------------------------------------------------------------------
Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.
--------------------------------------------------------------------------------


# Delete

You can remove the table by using the .drop_table() method.

In [12]:
db.drop_table()